In [1]:
#!pip install tensorflow==2.0.0-beta0

 El volumen de la unidad C no tiene etiqueta.
 El número de serie del volumen es: F8C5-003D

 Directorio de C:\Users\Remigio\Dropbox\9.Docencia\1.MATERIAS\3.APRENDIZAJE DE MAQUINA\2.Practicas\3.PracticaRedesNeuronalesProblemasClasificacion\CuadernosJupiter_Dataset

15/04/2020  10:34    <DIR>          .
15/04/2020  10:34    <DIR>          ..
13/04/2020  11:36    <DIR>          .ipynb_checkpoints
15/04/2020  01:09           349,061 RedNeuronalConTensorFlowKeras.html
15/04/2020  01:09            54,798 RedNeuronalConTensorFlowKeras.ipynb
15/04/2020  10:34             5,815 RedNeuronalSinLibrerias.ipynb
02/04/2020  20:57    <DIR>          titanic
07/10/2019  00:07            34,757 titanic.zip
               4 archivos        444,431 bytes
               4 dirs  923,140,694,016 bytes libres


In [3]:
#Esta guía explica el proceso para crear una arquitectura de redes neuronales, posteriormente explica el proceso de 
#optimización y fine tuning. Para ello, primeramente se define una red neuronal y se evalúa. Posteriormente, se ajustan
#los parámetros y se evalúa el modelo final.

#El dataset se llama Titanic, lo pueden encontrar en: 
#Dataset Kaggle: https://www.kaggle.com/c/titanic
#También lo adjunto en el directorio de esta guía

#Con este problema se intenta predecir si una persona sobrevivió o no sobrevivió, este tipo de problemas es aplicable a otros 
#problemas de sobrevivencia ante desastres.
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

df = pd.read_csv('master.csv')
df.head()

#Aquí solo con la vista podemos descartar muchas variables que son las siguientes:
#Nombre: No nos sirve porque no aporta información al resultado
#Ticket: Tampoco nos aporta Información
#Cabin: Podría aportarnos información pero tiene valores NaN (perdidos)
#PassangerID: Es el indice, no aporta información relevante
#Parch: Lo eliminaremos para tener una peor rendimiento para saber como optimizar el modelo en la siguiente sección

#Entonces nuestros datos de entrada “X” tendrán los siguientes valores:
#Sex: Genero de la persona
#Age: Edad de la persona
#SibSp: la cantidad de hermanos o esposa
#Fare: El impuesto que pagaron para embarcarse
#Embarked: Es la clase en donde se embarco la persona

,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [4]:
df = df.dropna(subset=['country', 'year', 'sex', 'age', 'suicides_no', 'population','suicides/100k pop','country-year','HDI for year','generation'])
df.head()
df.shape


(8364, 12)

In [5]:
#Seleccionamos las variables escogidas

Xsubset = df[['sex', 'age', 'suicides_no', 'population', 'suicides/100k pop']]
#Xsubset = df[['Pclass', 'Sex', 'Embarked']]
#Xsubset.fillna(0)

#para separar nuestra variable dependiente de la independiente, haremos lo siguiente:

y = df.generation.values
print(Xsubset)
type(Xsubset)

          sex          age  suicides_no  population  suicides/100k pop
72       male  25-34 years           13      232900               5.58
73       male  55-74 years            9      178000               5.06
74     female    75+ years            2       40800               4.90
75     female  15-24 years           13      283500               4.59
76       male  15-24 years           11      241200               4.56
...       ...          ...          ...         ...                ...
27815  female  35-54 years          107     3620833               2.96
27816  female    75+ years            9      348465               2.58
27817    male   5-14 years           60     2762158               2.17
27818  female   5-14 years           44     2631600               1.67
27819  female  55-74 years           21     1438935               1.46

[8364 rows x 5 columns]


pandas.core.frame.DataFrame

In [18]:

from sklearn import preprocessing
# Sample input labels
      
input_labels = Xsubset.iloc[:,1]
# Create label encoder and fit the labels
encoder = preprocessing.LabelEncoder()
encoder.fit(input_labels)

# Print the mapping 
#print("\nLabel mapping:")
#for i, item in enumerate(encoder.classes_):
#    print(item, '-->', i)
    
# Encode a set of labels using the encoder
test_labels =  Xsubset.iloc[:,1]
encoded_values = encoder.transform(test_labels)
#print("\nLabels =", test_labels)
#print("Encoded values =", list(encoded_values))
Xsubset.iloc[:,1] = list(encoded_values)
print(Xsubset)


          sex  age  suicides_no  population  suicides/100k pop
72       male    1           13      232900               5.58
73       male    4            9      178000               5.06
74     female    5            2       40800               4.90
75     female    0           13      283500               4.59
76       male    0           11      241200               4.56
...       ...  ...          ...         ...                ...
27815  female    2          107     3620833               2.96
27816  female    5            9      348465               2.58
27817    male    3           60     2762158               2.17
27818  female    3           44     2631600               1.67
27819  female    4           21     1438935               1.46

[8364 rows x 5 columns]


C:\Users\ACER\Anaconda3\envs\AmbientesRedesNeuronales\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [202]:
#Pre procesamiento

#Generalmente cuando entrenamos modelos de “Deep Learning” antes de pasar los datos, todos los valores numéricos hay que 
#normalizarlos, porque si tenemos valores muy altos y muy bajos en la misma tabla, al pasar por la función de activación, 
#dependiendo cual sea, no lanzará resultados óptimos, por lo que se necesita un escalado, para que los valores queden a 
#la misma escala, usaremos el StandarScaler, que es para valores, que tienen mucha diferencia entre ambos.

#Embarked, Sex, y PClass, son valores categóricos nominales, es decir, son valores que pertenecen a categorías, por ejemplo,
#PClass representa la clase en que estaban embarcados los pasajeros, sin embargo, los valores son nominales puesto que un
#pasajero de primera clase no representa mas que uno de segunda, a efectos prácticos, a nivel matemático, si lo tratamos como
#un numero no suma ni resta. Podemos tratarlo como una categoría, porque tenemos una cantidad fija de clases
#a lo largo de los datos. Lo mismo pasa con Embarked.

#Por ser valores categóricos nominales no podemos dar las categorías directamente, tenemos que realizar el proceso de Coding
#es decir, pasar las categorías en formato (one_hot), que consiste en poner tantos ceros como categorías, y para representar
#un valor se coloca un uno en la posición del valor, ejemplo:

#3 colores: azul, rojo, verde. la primera categoría corresponde a tantos ceros como tantas categorías que se tienen 
#azul [1,0,0]
#verde es el tercero, por lo que para representar ese valor irá un uno en esa posición
#verde = [0,0,1]

#Documentación ColumnTransformer: 
#https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html

#Creamos un Pipeline (un formateador o canal) de pre procesamiento con sk_learn, usaremos la función make_column_transformer
#Link para aprender más: https://scikit-learn.org/stable/modules/compose.html

#Los transformadores generalmente se combinan con clasificadores, regresores u otros estimadores para construir un 
#estimador compuesto. La herramienta más común es un Pipeline (tubería).

#Otro enlace con ejemplo: https://medium.com/vickdata/easier-machine-learning-with-the-new-column-transformer-from-scikit-learn-c2268ea9564c

In [34]:
#PRIMERA FORMA DE PREPROCESAR
Xsubset = Xsubset.dropna(subset=['age','suicides_no','population','suicides/100k pop','sex'])

preprocesador1 = make_column_transformer(
    (StandardScaler(),['age','suicides_no','population','suicides/100k pop']),
    (OneHotEncoder(),['sex']))

X = preprocesador1.fit_transform(Xsubset)
print(X.shape[1])
print(X.shape)

#print(X)

#print(preprocesador1)
categorical_features = ['sex']
cnamesDataset1 = ['age','suicides_no', 'population', 'suicides/100k pop']
cnamesDataset2 = preprocesador1.transformers_[1][1].get_feature_names(categorical_features)

#print(cnamesDataset2)
cnamesDataset1.extend(cnamesDataset2)
print(cnamesDataset1)

DatasetPreprocesado = pd.DataFrame(data=X,columns=cnamesDataset1)
print(DatasetPreprocesado.head())

DatasetPreprocesado.to_csv("MasterProcesado3.csv", sep=",",index = False) #sep es el separado, por defector es ","

6
(8364, 6)
['age', 'suicides_no', 'population', 'suicides/100k pop', 'sex_female', 'sex_male']
       age  suicides_no  population  suicides/100k pop  sex_female  sex_male
0 -0.87831    -0.283604   -0.407927          -0.369336         0.0       1.0
1  0.87831    -0.289478   -0.421758          -0.399288         0.0       1.0
2  1.46385    -0.299758   -0.456321          -0.408504         1.0       0.0
3 -1.46385    -0.283604   -0.395180          -0.426361         1.0       0.0
4 -1.46385    -0.286541   -0.405836          -0.428089         0.0       1.0


In [36]:
print(DatasetPreprocesado.head())
DatasetPreprocesado.extendencoded_values

       age  suicides_no  population  suicides/100k pop  sex_female  sex_male
0 -0.87831    -0.283604   -0.407927          -0.369336         0.0       1.0
1  0.87831    -0.289478   -0.421758          -0.399288         0.0       1.0
2  1.46385    -0.299758   -0.456321          -0.408504         1.0       0.0
3 -1.46385    -0.283604   -0.395180          -0.426361         1.0       0.0
4 -1.46385    -0.286541   -0.405836          -0.428089         0.0       1.0


NameError: name 'extendencoded_values' is not defined

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

#Ahora preparamos el perceptron. Importamos las neuronas simples y el modelo secuencial
#Modelo secuencial quiere decir que agregaremos capas y se conectarán de manera automática, 
#Dense es la librería de neuronas simples.
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import model_from_json
print('Librerías importadas')

Using TensorFlow backend.


Librerías importadas


In [15]:
X_train.shape[1]

6

In [235]:
#Construcción del Modelo o Arquitectura de Redes Neoronales
model = Sequential()

#La primera capa Dense recibe el numero de variables, que es la segunda dimensión de la matriz X, esto es X_train.shape[1]
#La primera capa tiene 32 neuronas. La función de activación es la función rectificadora.
model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
#La segunda capa tiene 64 neuronas. La función de activación es la función rectificadora.
model.add(Dense(64, activation='relu'))
#La capa de salida tiene 1 neurona. La capa de salida debe tener la misma dimensión como de cantidad de salidas queremos,
#por ejemplo, en este caso la salida "Survived" solo requiere 0 y 1. Puesto que 0 o 1 ocupan solo un valor dentro de cada dato,
#entonces 1 neurona es suficiente. La función de activación es sigmoide para clasificación por probabilidad.
model.add(Dense(1, activation='sigmoid'))

#Como tenemos dos posibles salidas "0 o 1", vamos a escoger que el error lo trate como una clasificación binaria, 
#el optimizador será nuestra función derivada que nos ayudará a determinar hacia donde mover los pesos.
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method

#imprimir arquitectura de la red
model.summary()

#Entrenamiento: 

#Entrenaremos por 100 epochs, el batch_size es un argumento importante, porque representa cada cuántos datos va a actualizar
#los pesos. Este es el método del gradiente descendiente estocástico que hace el proceso más eficiente y preciso.
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)
score = model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#mostrar pesos de la red
#print(model.get_weights())

#Guardar pesos y la arquitectura de la red en un archivo 

print("Guardando Red Neuronal en Archivo")  
# serializar modelo a JSON

# Guardar los Pesos (weights)
model.save_weights('model_weights.h5')

# Guardar la Arquitectura del modelo
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

print("Red Neuronal Grabada en Archivo")    
    
# Cargar la Arquitectura desde el archivo JSON
with open('model_architecture.json', 'r') as f:
    model2 = model_from_json(f.read())

# Cargar Pesos (weights) en el nuevo modelo
model2.load_weights('model_weights.h5')  

print("Red Neuronal Cargada desde Archivo") 

model2.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method
score = model2.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_55 (Dense)             (None, 32)                352       
_________________________________________________________________
dense_56 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_57 (Dense)             (None, 1)                 65        
Total params: 2,529
Trainable params: 2,529
Non-trainable params: 0
_________________________________________________________________
acc: 84.71%
Guardando Red Neuronal en Archivo
Red Neuronal Grabada en Archivo
Red Neuronal Cargada desde Archivo
acc: 84.71%


In [234]:
#Predicciones con nuevos datos

#La siguiente función tomará los datos de entrada, los va a integrar a una dataframe, los va a preprocesar, y 
#retornará una predicción con la salida "0" o "1", es decir, "sobrevivió" o "no sobrevivió"
def predict(Pclass=1, Sex='female', Age=60 ,Fare=0, Embarked='C'):
    cnames = ['Pclass', 'Sex', 'Age', 'Fare', 'Embarked']
    data = [[Pclass, Sex, Age, Fare, Embarked]]
    my_X = pd.DataFrame(data=data, columns=cnames)
    my_X = preprocesador.transform(my_X)
    return model.predict_classes(my_X)

print('Predicción:',predict(Age=32 ,Fare=9))
print('Predicción:',predict(Pclass=1, Sex='female', Age=60 ,Fare=0, Embarked='C'))

Predicción: [[1]]
Predicción: [[0]]


In [179]:
#Optimizacion y fine tuning

#Para darnos mejor idea de cuanto es nuestro error, realizaremos un proceso llamado "cross_validation", este proceso entrena
#la cantidad de veces que definamos y devuelve la métrica indicada para todos los pasos, esto es mas que nada porque en cada
#epoch la pérdida suele variar, entonces esto nos dará la precisión calculando la media de todas estas precisiones.

#El proceso de Optimizacion consiste en reducir el error. Es decir, buscamos que la precision sea más alta. Mientras
#mayor es el accuracy será mejor el modelo de red neuronal para este problema.

#El proceso de "Fine Tunning" consiste en buscar posibles errores, y combinaciones de parámetros que puedan mejorar el modelo
#Este proceso consume mucha memoria RAM, por lo tanto es recomendable usar alguna nube con mejores recursos a los locales, como
#por ejemplo: Google Colab. 
#Enlaces: 
#https://colab.research.google.com/notebooks/welcome.ipynb#recent=true
#https://colab.research.google.com/notebooks/

#Estrategia de Optimización: 
#1.Compilación, 
#2.Densidad de las capas de neuronas, y 
#3.Agregar una cantidad de Dropout.

#Dropout: basicamente lo que hace es apagar neuronas al azar con el fin de que las neuronas no se vuelvan tan dependientes de
#los datos, es decir que se entrenen mejor para evitar el overfitting, para ello importaremos nuestra capa de dropout.
#La capa dropout recibe como parámetro un numero entre 0 y 1 que representa el porcentaje de neuronas que va a desactivar en 
#esa capa, por el momento quedará en 0.1, luego optimizaremos ese valor.

#Con el modelo "GridSearchCV" podremos optimizar todos los parámetros

#importamos el algoritmo cross validator, y un wrapper que permitirá usar modelos de keras con scikit learn
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dropout

In [180]:
#Evaluación del modelo original: Esto puede tardar unos 30 segundos.
def build_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['acc'])
    return model

#El modelo se pasa como parámetro
estimator = KerasClassifier(build_fn=build_model, epochs=100, batch_size=64) 
#cv es la cantidad de veces de entrenamiento del modelo
#n_jobs es para ocupar mas de un procesador. El parámetro -1 indica que queremos utilizar todos los procesadores disponibles
accuracies=cross_val_score(estimator, X_train, y_train, cv=10, n_jobs=-1)
mean_acc=accuracies.mean()
print('Precision media: ', mean_acc)
##Como resultado al proceso de entrenamiento, tenemos un accuracy en promedio aprox. de: 0.4971. Intentaremos mejorarlo.
#Simplemente con optimizer "adam", el resultado aprox. es: 0.798. Pruebenlo. Cambien optimizer='adadelta' a optimizer='adam'

Precision media:  0.5481203019618988


In [ ]:
#1. Compilación: Prueba de mejores parámetros batch_size, epochs y optimizer
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.


def build_model(optimizer):
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['acc'])
    return model


#parámetros que queremos probar, y sus valores 
#probaremos con batch_size, epochs, y optimizador, con el fin de encontrar la mejor combinación entre estos tres parámetros.
parameters = parameters = {'batch_size': [16,32],
             'epochs':[100,500],
             'optimizer': ['adadelta', 'rmsprop']}

estimator = KerasClassifier(build_fn=build_model, verbose=0)
#Ahora no le pasamos los parámetros al KerasClasifier, porque se los pasaremos a través de GridSearchCV
#el argumento verbose=0 es para que no muestre salida, si lo dejamos en cero, no mostrará la barra de progreso del entrenamiento
#GridSearchCV: recibe como parámetros nuestro modelo, nuestros parámetros, la medida sobre la que queremos comparar, y la 
#cantidad de veces que lo entrenará para sacar la media de accuracy.
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)
#Un ejemplo de resultados es: {'batch_size': 16, 'epochs': 100, 'optimizer': 'rmsprop'}
#Esto indica que el optimizador "adadelta" no es adecuado. Y es que este optimizador NO sirve para este tipo de problemas.

In [ ]:
#2. Densidad de las capas de neuronas
#Notemos que se incluyen los mejores parámetros del paso de optimización anterior (batch_size, epochs y optimizer)
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(l1, l2):
    model = Sequential()
    model.add(Dense(l1, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'l1':[16,32,64,128,256],
                           'l2':[16,23,64,128,256]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)

#Resultados: {'l1': 32, 'l2': 16}
#Los resultados indican que hubo un error en la red original, las capas van desde la más densa, a la menos densa.

In [ ]:
#3. Proceso con dropouts: apagar un porcentaje de neuronas al azar con el fin de que las neuronas no se vuelvan tan 
#dependientes de los datos.
##Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(d1, d2):
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(d1))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(d2))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'d1':[0.1,0.2,0.3],
                            'd2':[0.1,0.2,0.3]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)

#Resultados: {'d1':0.2, 'd2':0.3}

In [12]:
#Evaluación del Modelo Final

#Finalmente, veamos como mejoró nuestro modelo, vamos a repetir el proceso de la validación cruzada.
#Esto puede probarse localmente. Ya con los mejores parámetros evaluamos la red neuronal. Puede tardar un minuto.
def build_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
accuracies = cross_val_score(estimator, X_train, y_train, cv=10, n_jobs=-1)
mean_acc = accuracies.mean()
variance_acc = accuracies.std()
print('Precision media: ', mean_acc)

#Y el resultado es:
#Precision media aprox.: 0.8067
#Pasamos de un 50% precisión a un 80% de precisión, por lo que se recomienda: 
#hacer siempre el proceso de fine tunning, porque ayudará a crear modelos correctos en la mayoría de los casos.

C:\Users\UPS\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Precision media:  0.8067355871200561
